In [1]:
%load_ext autoreload
%autoreload 2

import os, glob
import matplotlib.pyplot as plt
import numpy as np
import re
from tqdm import tqdm
import torch
from scipy.ndimage import gaussian_filter

from inxss import calc_Sqw_from_SpinW_results
from inxss.experiment import SimulatedExperiment

In [2]:
data = torch.load('/pscratch/sd/z/zhantao/inxs_steering/SpinW_data/summarized_AFM_data_2023Sep13.pt')

In [3]:
data.keys()

dict_keys(['Syy', 'Szz', 'S', 'w_grid', 'q_grid', 'params'])

In [5]:
grid_info = {
    'h_grid': [-2.0, 2.0, 121],
    'k_grid': [-2.0, 2.0, 121],
    'l_grid': [-10.0,10.0, 41],
    'w_grid': [20.0, 190.0, 86]
}


for key, val in grid_info.items():
    exec(f'{key}=np.linspace(*val)')
    
hklw_grid = np.moveaxis(np.stack(np.meshgrid(h_grid, k_grid, l_grid, w_grid, indexing='ij'), axis=0), 0, -1)

# grid_info['hklw_grid'] = torch.from_numpy(hklw_grid)

torch.save(
    grid_info, '/pscratch/sd/z/zhantao/inxs_steering/smoothed_SpinW_data/sigma_0.75/grid_metadata'
)

In [5]:
for idx in tqdm(range(data['params'].shape[0])):

    sim_exp = SimulatedExperiment(
        q_grid = data['q_grid'],
        w_grid = data['w_grid'],
        Syy_grid = data['Syy'][idx],
        Szz_grid = data['Szz'][idx]
    )

    sim_exp.prepare_experiment(torch.from_numpy(hklw_grid))

    sqw_sm = sim_exp.Sqw.numpy().copy()

    sqw_sm = gaussian_filter(sqw_sm, 0.75)

    _data_save = {
        'param': data['params'][idx],
        'S': torch.from_numpy(sqw_sm).float()
    }

    torch.save(
        _data_save, f'/pscratch/sd/z/zhantao/inxs_steering/smoothed_SpinW_data/sigma_0.75/sample_{idx:03d}.pt'
    )

100%|██████████| 600/600 [3:51:20<00:00, 23.13s/it]  
